# TUG Test Preprocessing and Feature Extraction

## Imports

In [2]:
import os
import json
import pandas as pd
import datetime
import numpy as np

from math import sqrt
from functools import reduce

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from segmentation import SEGMENTS

## Global variables

In [3]:
DATA_DIR = "DATA"
FEATURES_DIR = "FEATURES"

## MAX RESOLUTION OF DEVICE SENSOR.
ACC_MAX_RES = 78.4532
GYRO_MAX_RES = 34.906586

## Functions

### Data loading

In [6]:
def data_file_desc(data_file):
    file_path = os.path.join(DATA_DIR, data_file)
    file_desc = data_file.split("_")
    file_desc = "_".join(file_desc[0:2])
    
    return file_path, file_desc


def load_data_file(data_file_path):
    with open(data_file_path) as file:
        return json.load(file) 
    
    
def to_dataframe(records_file):
    tidy_records = []

    for records in records_file:
        for record in records['records']:
            tidy_records.append({
                "type": records['type'],
                "timestamp": record['timestamp'],
                "x": record['x'],
                "y": record['y'],
                "z": record['z']
            })
    df = pd.DataFrame(tidy_records)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    return df

###  Visualize the data

In [5]:
def plot_components(dataframe, components):
    sensor_types = dataframe.type.unique()
    n_subplots = len(sensor_types)
    fig = make_subplots(rows=n_subplots, cols=1, subplot_titles=sensor_types, shared_xaxes=True)
    
    for i, sensor_type in enumerate(dataframe.type.unique()):
        sensor_df = dataframe[dataframe.type == sensor_type]
        for component in components:
            fig.add_trace(
                go.Scatter(x=sensor_df.timestamp, y=sensor_df[component], name="{0} {1}".format(sensor_type, component)),
                row=i+1, col=1,
            )
        
    fig.update_layout(height=400*n_subplots)
    fig.update_yaxes(title_text="Magnitude")
    fig.show()
    


    
    
def plot_triaxial_components(dataframe, components_group=[['x_acc', 'y_acc', 'z_acc'], ['x_gyro', 'y_gyro', 'z_gyro']]):
    n_subplots = len(components_group)
    fig = make_subplots(rows=n_subplots, cols=1, shared_xaxes=True)
    
    for i, group in enumerate(components_group):
        for component in group:
            fig.add_trace(
                go.Scatter(x=dataframe.timestamp_acc, y=dataframe[component], name=component),
                row=i+1, col=1,
            )
            
    fig.update_layout(height=400*n_subplots)
    fig.update_yaxes(title_text="Magnitude")
    fig.show()
    

### Data Cleaning (Temporal alignment)

In [7]:
def split(dataframe, types=['ACCELEROMETER', 'GYROSCOPE']):
    splits = []
    for typ in types:
        splits.append(dataframe[dataframe.type == typ])
    return splits


def temporal_trim(df1, df2):
    def global_start():
        start_df1 = df1['timestamp'].min()
        start_df2 = df2['timestamp'].min()
        return max(start_df1, start_df2)
    
    def global_end():
        end_df1 = df1['timestamp'].max()
        end_df2 = df2['timestamp'].max()
        return min(end_df1, end_df2)
    
    def trim(df, start, end):
        start_thresh = start.replace(microsecond=start.microsecond - 1000)
        end_thresh = end.replace(microsecond=end.microsecond + 1000)
        return df[(df.timestamp >= start_thresh) & (df.timestamp <= end_thresh)]
    
    start = global_start()
    end = global_end()
    
    return trim(df1, start, end), trim(df2, start, end)
    

def merge(df_acc, df_gyro):
    def copy_reset(df):
        return df.copy().reset_index(drop=True)
    
    def drop(df, columns):
        return df.drop(columns, axis=1)
    
    df_acc = drop(copy_reset(df_acc), ['type'])
    df_gyro = drop(copy_reset(df_gyro), ['type'])
    
    df_merged = df_acc.join(df_gyro, lsuffix="_acc", rsuffix="_gyro")
    return df_merged.dropna()

### Data Labeling

In [8]:
def add_labels(desc, df):
    df_copy = df.copy()
    segments = SEGMENTS[desc]
    def compute_label(timestamp):
        time = timestamp.time()
        if time < segments['stand_start'] or time >= segments['sit_end']:
            return 'SIT'
        if segments['stand_start'] <= time < segments['stand_end']:
            return 'STANDING'
        if segments['turn1_start'] <= time < segments['turn1_end'] or segments['turn2_start'] <= time < segments['turn2_end']:
            return 'TURNING'
        if segments['sit_start'] <= time < segments['sit_end']:
            return 'SITTING'
        return 'WALKING' 
    
    df_copy['gt'] = df_copy.timestamp_acc.apply(compute_label)
    
    return df_copy

### Data Normalization

In [9]:
def norm(value, maxVal, minVal, ran=[-1, 1]):
    return (ran[1] - ran[0]) * (value - minVal) / (maxVal - minVal) + ran[0]


def normalize_acc(value):
    return norm(value, ACC_MAX_RES, -ACC_MAX_RES)
    
    
def normalize_gyro(value):
    return norm(value, GYRO_MAX_RES, -GYRO_MAX_RES)


def normalize(df):
    df_copy = df.copy()
    df_copy[['x_acc', 'y_acc', 'z_acc']] = df_copy[['x_acc', 'y_acc', 'z_acc']].apply(normalize_acc)
    df_copy[['x_gyro', 'y_gyro', 'z_gyro']] = df_copy[['x_gyro', 'y_gyro', 'z_gyro']].apply(normalize_gyro)
    return df_copy

### Feature extraction

In [ ]:
def windows(data, window_size, step):
    r = np.arange(len(data))
    s = r[::step]
    z = list(zip(s, s + window_size))
    f = '{0[0]}:{0[1]}'.format
    g = lambda step : data.iloc[step[0]:step[1]]
    return pd.concat(map(g, z), keys=map(f, z))

In [ ]:
def extract_features(wdf, original_shape):
    df = pd.DataFrame()
    for i in range(0, original_shape, STEP_SIZE):
        window = wdf.loc["{0}:{1}".format(i, i+WINDOW_SIZE)]
        #normalized_values= window[['x_acc_norm', 'y_acc_norm', 'z_acc_norm', 'x_gyro_norm', 'y_gyro_norm', 'z_gyro_norm']]
        normalized_values= window[['x_acc', 'y_acc', 'z_acc', 'x_gyro', 'y_gyro', 'z_gyro']]
        
        mean_s = normalized_values.mean(axis=0)
        median_s = normalized_values.median(axis=0)
        max_s = normalized_values.max(axis=0)
        min_s = normalized_values.min(axis=0)
        std_s = normalized_values.std(axis=0)
        range_s = max_s - min_s
        rms_s = rms_series(window)
        pitch_roll_s = pitch_and_roll(window)
        angle_s = compute_gyro_angle(window)
        
        ground_truth = compute_best_class(window)
        
        window_features = join_series([mean_s, median_s, max_s, min_s, std_s, range_s, rms_s, pitch_roll_s, angle_s, ground_truth],
                                      ["_mean", "_median", "_max", "_min", "_std", "_range", "", "", "", ""])
        
        df = pd.concat([df, window_features], ignore_index=True)
    return df


def join_series(series, suffixes):
    for i, serie in enumerate(series):
        current_df = serie.to_frame().transpose()
        if i == 0:
            df = current_df
        else:
            df = df.join(current_df, lsuffix=suffixes[i-1], rsuffix=suffixes[i])
    return df


def to_series(data, index):
    return pd.Series(data, index=index)
       
    
def rms_series(df):
    def rms(values):
        return sqrt(reduce(lambda prev, curr: prev + curr ** 2, values, 0) / len(values))
    
    return to_series([
        rms(df.x_acc), rms(df.y_acc), rms(df.z_acc),
        rms(df.x_gyro), rms(df.y_gyro), rms(df.z_gyro)
    ], ['x_acc_rms','y_acc_rms','z_acc_rms', 'x_gyro_rms','y_gyro_rms','z_gyro_rms'])
    
    
def pitch_and_roll(df):
    def angular_function(a, b):
        return np.arctan2(a, b) * 180/np.pi
    
    return to_series([
        angular_function(df.y_acc, df.z_acc).mean(),
        angular_function(-df.x_acc, np.sqrt(np.power(df.y_acc, 2) + np.power(df.z_acc, 2))).mean()
    ], ['pitch', 'roll'])
 
    
def integrator(series):
    return np.trapz(series)


def compute_gyro_angle(df):
    return to_series([integrator(df.x_gyro), integrator(df.y_gyro), integrator(df.z_gyro)], ['x_angle', 'y_angle', 'z_angle'])


def compute_best_class(df):
    ground_truth = df['gt'].value_counts()
    #if len(ground_truth) == 1 or ground_truth.max() > ground_truth.sum() * 3 / 4: #Only one class or 75% of samples are from majority class
    best = ground_truth.index[0]
    #else:
    #    best = 'MIXED' 
        
    return to_series([best], ['CLASS'])

## Play

### Load raw data

In [10]:
raw_data = {}

for data_file in os.listdir(DATA_DIR):
    file_path, file_desc = data_file_desc(data_file)
    if not os.path.isfile(file_path):
        continue
    records_file = load_data_file(file_path)
    raw_data[file_desc] = to_dataframe(records_file)

### Clean data

In [24]:
clean_data = {}

for desc, data in raw_data.items():
    df_acc, df_gyro = split(data)
    df_acc, df_gyro = temporal_trim(df_acc, df_gyro)    
    df_merged = merge(df_acc, df_gyro)
    
    clean_data[desc] = normalize(add_labels(desc, df_merged))

### Extract and save features

In [25]:
WINDOW_SIZE = 50
STEP_SIZE = WINDOW_SIZE // 2

features_df = pd.DataFrame()
for desc, data in clean_data.items():
    windowed_df = windows(data, WINDOW_SIZE, STEP_SIZE)
    window_features = extract_features(windowed_df, data.shape[0])
    features_df = pd.concat([features_df, window_features], ignore_index=True)
    
features_df.to_csv(os.path.join(FEATURES_DIR, 'features_full_norm_01.csv'), index=False)